#### 토크나이저 및 모델 초기화

In [1]:
import torch
from transformers import AutoTokenizer, BartForConditionalGeneration, AutoModel

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") # gpu
tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2") # tokenizer
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2").to(device) # model

C:\Anaconda3\envs\torch-py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


#### 데이터셋 불러오기

In [2]:
import os
import re
import json
import datasets
import pandas as pd
from glob import glob
from datasets import load_dataset

**Train data**

In [3]:
summaries = []
passages = []

# 두 개의 폴더에 나눠져 있는 데이터를 리스트에 모은다
## 2~3sent
for f_name in glob(os.path.join("D:/jupyter/story-generation/data/story/summarization/train","literature/2~3sent/*.json")) :
    with open(f_name, 'r',encoding="utf-8") as f:
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])
        
## 20per
for f_name in glob(os.path.join("D:/jupyter/story-generation/data/story/summarization/train" ,"literature/20per/*.json")) :
    with open(f_name, 'r',encoding="utf-8") as f:
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])

# 한자 제거
passages = [re.sub("\([^\(\)]+\)", "", passage) for passage in passages]

train_df = pd.DataFrame([ x for x in zip(summaries, passages)], columns = ["input_texts","target_texts"])
train_df.head()

,input_texts,target_texts
0,아내에게 돈이 있을지 모른다는 귀띔을 문 서방은 믿을 수 없었으나 아내에게는 역시 ...,그러나 그보다도 절통한 것은 아내의 어리석음에서 생긴 비극이었다. 문 서방이 한이...
1,남 주사와 일지매는 승강기 맨 앞에 서서 윗층으로 올라가게 되었으나 해룡피 외투는 ...,일지매는 그 신사의 뒷모양이 선듯 눈에 띄자 공연히 가슴이 울렁하며 한 치각이나 아...
2,화증이 나는 것을 참고 삯군을 서넛이구 사보라 했더니 들은 성도 않고 걸레질만 치다...,"“삯군을 서넛이구 좀 사보게?” 화증이 나는 것을 짐짓 참고, 순순히 말로 일렀다..."
3,농익은 연시와 같이 붉은 뺨의 처녀들이 웃고 지껄이며 지나간다.,어느새 임해전 길고 넓은 복도와 뜰엔 사람의 사태다. 활짝 열어 제친 궁문 으로 ...
4,미란이가 동경 여행 이야기를 할 때 울적한 표정으로 조용히 앉아있던 단주에게 왜 말...,단주는 주인이 돌아온 이상 자기의 직책은 다한 듯 다음날부터 제대로 아 파트로 돌...


**Valid Data**

In [4]:
summaries = []
passages = []

# 두 개의 폴더에 나눠져 있는 데이터를 리스트에 모은다
## 2~3sent
for f_name in glob(os.path.join("D:/jupyter/story-generation/data/story/summarization/valid","literature/2~3sent/*.json")) :
    with open(f_name, 'r',encoding="utf-8") as f:
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])
        
## 20per
for f_name in glob(os.path.join("D:/jupyter/story-generation/data/story/summarization/valid" ,"literature/20per/*.json")) :
    with open(f_name, 'r',encoding="utf-8") as f:
        json_data = json.load(f)
        passages.append(json_data['Meta(Refine)']['passage'])
        summaries.append(json_data['Annotation']['summary1'])

# 한자 제거
passages = [re.sub("\([^\(\)]+\)", "", passage) for passage in passages]

valid_df = pd.DataFrame([ x for x in zip(summaries, passages)], columns = ["input_texts","target_texts"])
valid_df.head()

,input_texts,target_texts
0,동경으로 떠나게 된 실은 눈을 붉히며 떠날 용기가 나지 않아서 떠나는 날짜를 흐려오...,“이 걱정쟁이 같으니 누굴 칠면조나 카멜레온으로 아나부다.” “저 없는 동안에 모...
1,입내할 채비를 갖추고 기다리고 있던 대원군은 날이 어둔 후 창덕궁으로 들어갔다.,"그에 대조되어, 세도하고 행학하던 사람들은 벼락을 맞은 듯 숨어 들어가 몸을 떨었..."
2,어머니께 가면 항상 친절하게 잘해주셨지만 아파트의 사무원은 그만두라는 말을 항상 하...,"“면도를 빌려드릴까요?” 그러니까 사내는 머리를 극적극적 긁으며, “에이 뭐 면도..."
3,어둠 속에서도 자태는 또렷했고 충동적으로 몸은 쏠리었는데 그 순간 눈을 굴린 것은 ...,"야릇한 방, 페페의 정성, 준비된 식탁, 갸비이의 호기심, 페페의 열정 ─ 두 사람..."
4,흥식이는 옥지의 말을 너무도 신용하는 우식이가 미워서 혼사 준비에 대해 말을 하지 ...,과연 틀림이 없다. 백작의 집에서는 혼사 준비에 골몰하는 것을 볼진대 옥 지의 2...


In [5]:
train_df.to_csv("train_data.csv", index=False)
valid_df.to_csv("valid_data.csv", index=False)

**Dataset**

In [6]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

In [7]:
print('[ Train ]\n')
print('input : ', train_dataset['input_texts'][300])
print('target : ', train_dataset['target_texts'][300])

[ Train ]

input :  자기의 잘못으로 박 흥식에게 위협을 당한 김 정자는 분해서 가슴이 터질 지경이다.
target :   박 흥식에게 위협을 당한 김 정자는 분한 마음이 폭발하여 가슴이 터질 지 경이다. 그러나, 도무지 자기의 잘못인즉 누구를 원망하리요. 허영에 침혹 된 여자란 이같은 일이 한두 번이 아닐 것이다. 전번에는 별별 비루한 행동을 다하고 할 소리 못 할 소리를 다하면서 사 랑하느니 무엇하느니 하던 김 정자도 일시 허영으로 인하여 김 자작집으로 시집을 가버렸다. 이같은 창피한 경우를 당한 박 흥식의 분풀이도 사실로 말하면 당연한 일이라고 할 수 밖에 없다. 그러나, 김 정자는 무슨 염치와 무슨 면목으로 그같이 뻔뻔한지 도무지 흥 식의 하는 말을 듣지 않으려고만 한다. 그러다가 필경 흥식의 최후 수단으 로 그 편지를 자기 남편되는 김 자작에게 보이겠다는 말에는 정자 부인도 놀라지 않을 수 없었다. 그 까닭에 자기의 가장 사랑하던 진주목걸이까지 내어놓으려고 했다. 그러나, 흥식이는 오히려 만족히 알지 못하는 모양이다. 그리하여 두 사람 사이에는 또 다시 정론이 일어났다. 흥식은, 정자 씨, 그 손에 낀 반지도 내어놓으시오. 아마 그것은 값 나갈 듯하 니…… 다른 것은 다 내어놓아도 이것은 못 내놓겠읍니다. 왜 그래요? 그러면 어떻게 하신다는 말씀이오? 그것은 결혼 반지입니다. 남작이 손수 끼워 준 것이니까 여자가 되어 결 혼 반지를 빼앗기면 이혼한 것이나 다를 것이 무엇입니까? 흥! 말은 좋소. 결혼 반지? 내게서도 이전에 그같은 반지를 받았지요? 그리고 또 이것은 누구에게서 받은 반지란 말이오? 이 말에는 정자 부인으로도 아무 대답이 없었다.



In [8]:
print('[ Validation ]\n')
print('input : ', valid_dataset['input_texts'][300])
print('target : ', valid_dataset['target_texts'][300])

[ Validation ]

input :  꿈은 젊은이의 양식이라지만 놀음을 갔다가도 그 꿈이 떠오르면 모든 게 시들해지고 그야말로 기적이었다.
target :   ─ 그것은 짤막한 놀음이었어요. 두 시간도 채 못됐던가 해요. 아마 작정 이 두 시간이었던 모양인데 지금 생각하니 시간이 지날까봐 그것을 겁냈던 것 같아요. 그러기에 제 시간도 다 채우지 못하고 주춤주춤 일어났던 게 죠… 아마 그것이 기생생활 오륙 년 동안에 가장 인상깊었던 놀음인가 해요. 무엇이라고 설명키는 어려워도 가슴속에 무슨 뭉클한 감정이 며칠을 두고 남겠지요. 그중에서도 그 이야기를 하던 젊은이의 인상이 얼마를 두고 머리에서 사라 지지 않습디다. 그날 밤 그 자리가 아마 내게는 기적이었던가 해요. 바로 그날 밤이었어 요. 나는 그 사람의 꿈을 꾸었어요. 그 우렁차고 부드러운 말소리로 내 귀 에다 대고, “취향이, 이런 생활이 그렇게도 좋단 말이오. 달리 방도를 차리시오. 기 생생활에 견딜 만한 노력을 아끼지 않는다면 굶어 죽기야 하겠소?” 나는 이렇게 대답하였더라오. “내게는 그런 기적이 없었답니다.” “기적? 그러나 기적을 기다리는 것은 너무 소극적이오… 기적을 기다려서 는 안되오. 자진해서 그 기적을 만드시오!” 아우님! 꿈이란 할 수 없더군요. 바로 요 며칠 전에 잡지에선가 “꿈은 젊 은이의 양식”이란 말이 있습디다만 꿈이란 젊은이에게 있어선 안될 것이 라고 난 생각해요! 뭐야요? 글쎄, 그 꿈이란 말이 이상이라는 의미라면 모 르겠지마는… 어쨌든 그 꿈은 얼마를 두고 나를 울렸어요! 놀음을 갔다가도 문득 그 꿈 생각이 나면 그대로 모든 게 다 시들해지구… 그것이야말로 기적이었어요. 적어도 내게 있어선 더없이 기묘한 기적이었 던 것을… 박복한 취향이는 그 기적을 꼭 잡지 못했군요.



**Processing**

In [9]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["input_texts"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_texts"], max_length=max_target_length, truncation=True,padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
# 입력 문장과 타켓 문장 토큰의 최대 토큰 길이 지정
max_input_length = 64
max_target_length = 512

In [11]:
#train_data =train_dataset.map(preprocess_function, batched=True)
#eval_data =eval_dataset.map(preprocess_function, batched=True)
#train_data = train_data[['input_ids','attention_mask','label']]

train_data =train_dataset.map(preprocess_function, batched=True).remove_columns(["input_texts","target_texts","token_type_ids"])
valid_data =valid_dataset.map(preprocess_function, batched=True).remove_columns(["input_texts","target_texts","token_type_ids"])


print(train_data)
print(valid_data)

Parameter 'function'=<function preprocess_function at 0x00000211AAEAD8B8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.96ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9600
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1200
})


#### 모델 훈련 준비

**Arguments**

In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer
from transformers import DataCollatorForSeq2Seq, get_cosine_schedule_with_warmup

batch_size = 4

training_args = Seq2SeqTrainingArguments(
    output_dir=".results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    do_train=True,
    do_eval=True,
#    fp16=True,
    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

**Metric (평가 함수)**

In [13]:
import nltk
import numpy as np
from datasets import load_metric

nltk.download("punkt", quiet=True)
metric = load_metric("rouge")


# 문장을 줄바꿈으로 구분하는 함수
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

# Trainer에 전달할 평가 함수
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    # 디코딩
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) # 생성 결과 텍스트 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # 레이블 내 -100 교체
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # 레이블 텍스트 디코딩

    # 각 문장을 개행문자로 구분 (ROUGE는 각 문장 다음에 개행문자를 요구한다.)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    # ROUGE 점수 계산
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # 중간 점수 (median) 추출
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset = train_data,
    eval_dataset = valid_data,
    tokenizer=tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

C:\Anaconda3\envs\torch-py38\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9600
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4800


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.403600,3.292727,0.926100,0.241700,0.911500,0.907700,20.000000
2,3.289800,3.261404,0.852600,0.247200,0.845200,0.851700,20.000000


Saving model checkpoint to .results\checkpoint-500
Configuration saved in .results\checkpoint-500\config.json
Model weights saved in .results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in .results\checkpoint-500\tokenizer_config.json
Special tokens file saved in .results\checkpoint-500\special_tokens_map.json
Deleting older checkpoint [.results\checkpoint-23000] due to args.save_total_limit
Saving model checkpoint to .results\checkpoint-1000
Configuration saved in .results\checkpoint-1000\config.json
Model weights saved in .results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in .results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in .results\checkpoint-1000\special_tokens_map.json
Deleting older checkpoint [.results\checkpoint-23500] due to args.save_total_limit
Saving model checkpoint to .results\checkpoint-1500
Configuration saved in .results\checkpoint-1500\config.json
Model weights saved in .results\checkpoint-1500\pytorch_mod

TrainOutput(global_step=4800, training_loss=3.4262604904174805, metrics={'train_runtime': 1418.0734, 'train_samples_per_second': 13.539, 'train_steps_per_second': 3.385, 'total_flos': 731683749888000.0, 'train_loss': 3.4262604904174805, 'epoch': 2.0})

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1200
  Batch size = 4


{'eval_loss': 3.2614035606384277,
 'eval_rouge1': 0.8526,
 'eval_rouge2': 0.2472,
 'eval_rougeL': 0.8452,
 'eval_rougeLsum': 0.8517,
 'eval_gen_len': 20.0,
 'eval_runtime': 66.2722,
 'eval_samples_per_second': 18.107,
 'eval_steps_per_second': 4.527,
 'epoch': 2.0}

#### 인퍼런스

**체크포인트 로드**

In [ ]:
#load ckpt

ForConditionalGeneration.from_pretrained(".results/checkpoint-12000").to(device)

**인퍼런스 함수**

In [16]:
def generate_output(test_samples, model):
    inputs = tokenizer(
        test_samples["input_texts"],
        padding="max_length",
        truncation=True,
        max_length=max_target_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids,
                             max_length = 256,
                             min_length=32,
                             top_p = 0.92,
                             num_beams=5,
                             no_repeat_ngram_size=2,
                             attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


valid_data_t = valid_dataset.map(preprocess_function, batched=True)

test_samples = valid_data_t.select(range(10))

#model_before_tuning = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2")


#summaries_before_tuning = generate_output(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_output(test_samples, model)[1]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.22ba/s]


In [17]:
from tabulate import tabulate

print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,           
#            summaries_before_tuning,
        ),
#        headers=["Id", "Output (Fine-tuned)", "Output (Zero-tuning)"],
        headers=["Id", "Output (Fine-tuned)"],
    )
)
print("\n Korean BART Fine-tuning Output:\n")

print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            list(test_samples["input_texts"]),            
            list(test_samples["target_texts"]), 
        ),
    headers=["Id", "Input Text","Target Text"]
    
    )
)


print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["input_texts"])), headers=["Id", "Document"]))

  Id  Output (Fine-tuned)
----  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
summaries_after_tuning[9]

'영솔장의 지시에 따라 모인 사람들은 모두 총을 어깨에 둘러메고 무덤 일곱이 가지런히 놓인 곳으로 걸어갔다. 그 가운데서 가장 나이 많고 경험 많은 영 솔장이었다. 그도 역시 나이 많고, 경험 많았다. 그이들은 모두 자기네들끼리 모여서 그네들의 무덤을 둘러보았다. 이윽고 그 무덤의 일곱은 가지가지 흩어져서 흩어졌다. 그 중에도 가장 나이가 많고 경험이 많은 이들도 있었다. 모두들 그 네네의 무덤에 모여들었다. 이 일곱을 가리켜서 무덤이라고 하였다. 이 다섯은 모두 나이 많은 사람들이었다. 그리고 그 중에 한 사람씩은 나이 어린 사람이 있었다. 그리고 한 사람은 나이가 많은 사람이었다. 그리고 또 한 사람이 나이 든 사람이 있다. 그 사람은 나이 먹은 사람이다. 나이든 사람이면 누구나 다 나이와 경험이 많다. 그리고 나이가 많으면 많을수록 경험도 많아진다. 그러나 나이가 든 사람은 그 나이보다 훨씬 경험이 적다. 나이가 많아서 경험은 적지만 경험의 많고 적음은 그보다 더한 것이 없다. 나이 적은 사람은 경험에 비하여 경험이 적은 사람이다.\n그는 나이가 들수록 경험이 많아지는 것을 알았다. 나이 많아도 경험에는 많은 것이 있는 것이다. 경험이란 경험이다. 경험이라는 것은 경험이라고 할 수 없다. 경험이라 함은 경험으로'

In [23]:
inputs = tokenizer(
        "어떤 남자가 빨간 오토바이를 타고 있다.",
        padding="max_length",
        truncation=True,
        max_length=max_target_length,
        return_tensors="pt",
    )
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)

outputs = model.generate(input_ids,
                             max_length = 700,
                             min_length=32,
                             top_p = 0.92,
                             num_beams=5,
                             no_repeat_ngram_size=2,
                             attention_mask=attention_mask)
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

output_str[0]

'어떤 남자가 남자가 빨간 오토바이를 타고 있는 것을 보았다. 어떤 남자는 빨간 자전거를 타고 있다. 어떤 여자는 빨간 고무보트를 타고 있고 어떤 사람은 빨간 자동차를 탄다. 어떤 남성은 빨간 바지를 입은 채, 어떤 여자와 나란히 서서, 어떤 여자의 뒤를 따라가며, 어떤 여자가 어떤 남자의 뒤를 따르고 있는 것이다. 어떤 남자와 어떤 여자를 사이에 두고 그 여자는 그 여자의 옆으로 가면서, “여보!” 하고 부르짖는다. “어디서 오셨어요?” “오라버니?” “아니오, 어디 갔다 왔어요.” “그러면 오시겠읍니까?” “이리 오십시오.” “어딜 가셔요.” 어떤 여편네는 이렇게 대답한다. “저기 오셔요! 어디 가세요? 어디 있셔요? 어디 있어요? 오시오. 어디 있지요?......” 그 여편의 대답은 의외로 간단하였다. “무슨 일인지 모르겠읍니다.” 이 여 편네의 대답이 의외의 대답을 하였다. “그런데 오시는 분은 어디 계십니까?.......” “한 번 가보세요.” 하는 여인의 대답이었다. “네, 가십시다.” 여자는 대답하였다. “그렇습니다.” 남편은 고개를 끄덕끄덕하며 “왜? ××가 오신단 말이오? 그까짓 게 오라 버렸단 말이지요. ─” 나는 대답하지 않았다. “그럼, 그놈이 오냐. 오냐, 그 놈이 왔다 갔어.” 여자의 대답에 남 편은 얼른 대답하였다.\n“오냐, 이놈의 놈을 어떻게 해. 이 놈의놈을 죽여 버리겠소.” 그는 이렇게 말하였다. 그러나 그는 대답할 말이 없었다. “나도 오겠소이다.” 그의 대답에는 아무 대답도 없었다. 그는 다시 말을 계속하였다. “어떤 놈은 오느냐? 저놈은 왜 오냐?” 그러나 그 남자는 대답하는 대신에 그 여자를 쳐다보았다. “그것은 내가 오라고 한 것이 아니오. 나는 오라는 것이오. 오라, 오라. 오, 오! 오오! ── 오오.” 두 사람은 서로 마주 앉았다. 두 사람의 눈은 마주 마주 섰다. 두 사람 사이에는 무슨 말이 오고 가는 듯하였다. 그 사람의 얼굴에는 웃음이 흘렀다. 그러나 그의 눈에는 눈물이 맺혔다. 그 눈에서는 눈물이 글썽글썽

In [26]:
inputs = tokenizer(
        "운동장에 모여 있는 사람들이 줄넘기를 하고 있다.",
        padding="max_length",
        truncation=True,
        max_length=max_target_length,
        return_tensors="pt",
    )
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)

outputs = model.generate(input_ids,
                             max_length = 512,
                             min_length=32,
                             top_p = 0.92,
                             num_beams=5,
                             no_repeat_ngram_size=2,
                             attention_mask=attention_mask)
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

output_str[0]

'운동장에 모여 있는 사람들이 줄넘기를 하고 있다. 운동장에는 사람들이 모여 있다. 그 중에 한 사람씩은 줄을 긋고 있는 것이 보이고, 또 다른 한 사람은 줄을 그은 것이 보인다. 그 중에도 한 사람이 줄을 당기고 있는 것도 보이며, 또 한 사람의 다른 사람이 줄 넘기를 하는 것도 보 았 다. 그 중에는 한 사람도 줄을 이르고 있는 사람이 보이기도 하고, 또 어떤 이는 줄을 당기기도 하는 사람도 보이기는 보이는데 그 가운데에 한 사람, 또 그 옆에 서 있는 사람도 보이는 사람이 보이는 사람인 것 같다. 그리고 그 옆에는 다른 사람의 그림자가 보이는데, 그 그림자의 그림자는 그림자 그림자에 비추인 듯도 하다. 이 그림자를 보고 있는 사람은 누구인지 알 수 없으나 그림의 그림자도 보이지 않는다. 그림자와 그림자로 보이는 사람은 그림자인 줄만 알 뿐이다. 그림자들은 그림자에게 비치인 줄은 알 수가 없다. 그림자들이 그림자들의 그림자들을 보고 있노라니 그림에 비치는 그림은 그림이 아닌 줄도 모른다. 그림을 그리는 사람들은 그림 속에 비친 그림자나 그림같이 보이는 줄로만 보이는 줄을 안다.\n그림자가 보이지 않 는 줄에는 그림도 비치지 않는 줄에 서는 그림들이 있다. 그림과 그림 사이에 있는 그림자들. 그림들은 그림으로 그려진 그림이다. 그림을 그리던 사람들은 줄에서 내려서 줄 위에 올라서서 그림이나 그릴 수 있는 줄의 그림을 그려 보았다. 그림 속에는 그림 한 장이 그려져 있다.\n그림자를 그리지 않는 것은 그림뿐이 아니다. 그림에서 나오는 그림들도 그림일 것이다. 그림 속에서 그려지는 그림들. 그것은 그림인 줄을 모르는 줄 알면서도 그림이라고는 생각지 않는 그림이었다. 그림 속의 그림들을 그렸다 그렸다고도 생각되지 않는 것이 그림이었 다. 그림 가운데서 그림을 그리고 있는 사람이나 그림을 그린 사람이나, 그림 속 그림들과는 거리가 먼 줄 속에 있는 사람들, 그림속에 그려 있는 그림을 보는 사람 등등 은 그림속의 그림 같은 줄거리를 그렸던 것이 분명하다. 그림속 그림